### extract metrics from raw data

In [ ]:
import pandas as pd
from datetime import datetime,timezone

In [32]:
df = pd.read_csv('data/drupal_raw.csv',index_col='qid')
df.index.is_unique

True

In [33]:
df.qviews = df.qviews.apply(lambda x: x.replace('k','000') if 'k' in x else x)
df.qviews = df.qviews.astype('int64')

In [34]:
df.qasktime = pd.to_datetime(df.qasktime)
df.qactivetime = pd.to_datetime(df.qactivetime)
df.responsetime = pd.to_datetime(df.responsetime)
df.dtypes

avotesum                      int64
qactivetime     datetime64[ns, UTC]
qanswers                      int64
qasktime        datetime64[ns, UTC]
qviews                        int64
qvotes                        int64
responsetime    datetime64[ns, UTC]
dtype: object

In [35]:
start_time = datetime(2020,1,1,tzinfo=timezone.utc)
end_time = datetime(2020,6,30,tzinfo=timezone.utc)
df = df[(df.qasktime>start_time)&(df.qasktime<end_time)]

In [36]:
df['activity'] = df['qvotes'].apply(lambda x:abs(x))+df['qanswers']+df['avotesum']
df['exposure'] = df['qviews']

In [37]:
df['dayofweek'] = df['qasktime'].dt.dayofweek

In [38]:
df['endurance'] = df.qactivetime-df.qasktime
df['endurance'] = df['endurance'].dt.total_seconds()
df = df[~(df['endurance']<0)]

In [39]:
df['response'] = df.responsetime-df.qasktime
df['response'] = df['response'].dt.total_seconds()
df = df[~(df['response']<0)]

In [40]:
df.to_csv('./data/drupal.csv',columns=['activity','exposure','dayofweek','response','endurance'])
df.describe()

,avotesum,qanswers,qviews,qvotes,activity,exposure,dayofweek,endurance,response
count,228.000000,228.000000,228.000000,228.000000,228.00000,228.000000,228.000000,2.280000e+02,1.660000e+02
mean,0.434211,0.912281,256.127193,0.333333,1.75000,256.127193,2.521930,2.891385e+06,1.449881e+06
std,0.890511,0.715608,343.711738,0.781579,1.66163,343.711738,1.913194,8.541088e+06,5.783982e+06
min,0.000000,0.000000,13.000000,-4.000000,0.00000,13.000000,0.000000,0.000000e+00,1.920000e+02
25%,0.000000,0.000000,53.000000,0.000000,1.00000,53.000000,1.000000,4.249500e+03,6.120750e+03
50%,0.000000,1.000000,110.000000,0.000000,1.00000,110.000000,2.500000,7.331950e+04,7.421300e+04
75%,1.000000,1.000000,347.500000,1.000000,2.00000,347.500000,4.000000,5.142810e+05,4.082222e+05
max,7.000000,4.000000,2000.000000,4.000000,12.00000,2000.000000,6.000000,5.485926e+07,5.277282e+07
